<a href="https://colab.research.google.com/github/nicekhan23/esp32_edge_ai_benchmark/blob/main/train_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import json
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

# ============================================================================
# DATA PREPARATION - For CSV with 5000 windows of each waveform
# ============================================================================

print("Loading data from CSV...")
df = pd.read_csv('all_waveforms_combined.csv')  # Your CSV file with 1000 samples per waveform

# Extract features and labels
print("Processing features and labels...")
X = df.drop(['label', 'window_index'], axis=1).values
y = df['label'].values

# Normalize data (0-4095 to 0-1 range)
X = X / 4095.0

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data: 70% train, 15% validation, 15% test
print("Splitting data...")
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Create directory structure
data_dir = Path("esp32_training_data/processed")
data_dir.mkdir(parents=True, exist_ok=True)

# Save processed data
print("Saving processed data...")
np.save(data_dir / "X_train.npy", X_train)
np.save(data_dir / "X_val.npy", X_val)
np.save(data_dir / "X_test.npy", X_test)
np.savetxt(data_dir / "y_train.txt", y_train, fmt='%d')
np.savetxt(data_dir / "y_val.txt", y_val, fmt='%d')
np.savetxt(data_dir / "y_test.txt", y_test, fmt='%d')

# Save label encoder classes
with open(data_dir / "label_classes.json", 'w') as f:
    json.dump(label_encoder.classes_.tolist(), f)

print(f"\nDataset Statistics:")
print(f"Total samples: {len(X)}")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test samples: {len(X_test)}")
print(f"Classes: {label_encoder.classes_}")

# ============================================================================
# CNN MODEL (Float32 and INT8)
# ============================================================================

def create_cnn_model(input_shape, num_classes, quantized=False):
    """Create a CNN model for waveform classification"""
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(2),

        tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(2),

        tf.keras.layers.Conv1D(128, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling1D(),

        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Train Float32 CNN model
print("\n" + "="*50)
print("Training CNN Float32 model...")
print("="*50)

model_f32 = create_cnn_model((256, 1), len(label_encoder.classes_))
model_f32.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Reshape data for CNN (add channel dimension)
X_train_cnn = X_train.reshape(-1, 256, 1)
X_val_cnn = X_val.reshape(-1, 256, 1)
X_test_cnn = X_test.reshape(-1, 256, 1)

history_f32 = model_f32.fit(
    X_train_cnn, y_train,
    validation_data=(X_val_cnn, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=1
        )
    ],
    verbose=1
)

# Evaluate CNN
cnn_test_loss, cnn_test_acc = model_f32.evaluate(X_test_cnn, y_test, verbose=0)
print(f"\nCNN Test Accuracy: {cnn_test_acc:.4f}")

# Save model
model_f32.save("cnn_float32_model.h5")
print("CNN Float32 model saved as 'cnn_float32_model.h5'")

# Convert CNN to INT8
print("\nConverting CNN to INT8...")

def representative_dataset_cnn():
    for i in range(100):  # Use 100 samples for calibration
        yield [X_train_cnn[i:i+1].astype(np.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model_f32)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_cnn
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

try:
    tflite_model = converter.convert()
    with open('cnn_int8_model.tflite', 'wb') as f:
        f.write(tflite_model)
    print("CNN INT8 model saved as 'cnn_int8_model.tflite'")

    # Get model size
    tflite_size = len(tflite_model) / 1024
    print(f"CNN INT8 Model size: {tflite_size:.2f} KB")
except Exception as e:
    print(f"Error converting CNN to INT8: {e}")

# ============================================================================
# MLP MODEL (Multi-Layer Perceptron)
# ============================================================================

def create_mlp_model(input_shape, num_classes):
    """Create an MLP model for waveform classification"""
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_shape,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Train Float32 MLP model
print("\n" + "="*50)
print("Training MLP Float32 model...")
print("="*50)

model_mlp_f32 = create_mlp_model(256, len(label_encoder.classes_))
model_mlp_f32.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_mlp_f32 = model_mlp_f32.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=1
        )
    ],
    verbose=1
)

# Evaluate MLP
mlp_test_loss, mlp_test_acc = model_mlp_f32.evaluate(X_test, y_test, verbose=0)
print(f"\nMLP Test Accuracy: {mlp_test_acc:.4f}")

# Save model
model_mlp_f32.save("mlp_float32_model.h5")
print("MLP Float32 model saved as 'mlp_float32_model.h5'")

# Convert MLP to INT8
print("\nConverting MLP to INT8...")

def representative_dataset_mlp():
    for i in range(100):
        yield [X_train[i:i+1].astype(np.float32)]

converter_mlp = tf.lite.TFLiteConverter.from_keras_model(model_mlp_f32)
converter_mlp.optimizations = [tf.lite.Optimize.DEFAULT]
converter_mlp.representative_dataset = representative_dataset_mlp
converter_mlp.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_mlp.inference_input_type = tf.int8
converter_mlp.inference_output_type = tf.int8

try:
    tflite_model_mlp = converter_mlp.convert()
    with open('mlp_int8_model.tflite', 'wb') as f:
        f.write(tflite_model_mlp)
    print("MLP INT8 model saved as 'mlp_int8_model.tflite'")

    tflite_size_mlp = len(tflite_model_mlp) / 1024
    print(f"MLP INT8 Model size: {tflite_size_mlp:.2f} KB")
except Exception as e:
    print(f"Error converting MLP to INT8: {e}")

# ============================================================================
# HYBRID MODEL (CNN + MLP)
# ============================================================================

def create_hybrid_model(input_shape, num_classes):
    """Create a hybrid CNN+MLP model"""
    inputs = tf.keras.layers.Input(shape=input_shape)

    # CNN branch
    x = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)

    x = tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # MLP branch (direct features)
    y = tf.keras.layers.Dense(64, activation='relu')(tf.keras.layers.Flatten()(inputs))
    y = tf.keras.layers.BatchNormalization()(y)

    # Combine branches
    combined = tf.keras.layers.Concatenate()([x, y])
    combined = tf.keras.layers.Dense(64, activation='relu')(combined)
    combined = tf.keras.layers.Dropout(0.3)(combined)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(combined)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

# Train Float32 Hybrid model
print("\n" + "="*50)
print("Training Hybrid Float32 model...")
print("="*50)

model_hybrid_f32 = create_hybrid_model((256, 1), len(label_encoder.classes_))
model_hybrid_f32.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_hybrid_f32 = model_hybrid_f32.fit(
    X_train_cnn, y_train,
    validation_data=(X_val_cnn, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=1
        )
    ],
    verbose=1
)

# Evaluate Hybrid
hybrid_test_loss, hybrid_test_acc = model_hybrid_f32.evaluate(X_test_cnn, y_test, verbose=0)
print(f"\nHybrid Test Accuracy: {hybrid_test_acc:.4f}")

# Save model
model_hybrid_f32.save("hybrid_float32_model.h5")
print("Hybrid Float32 model saved as 'hybrid_float32_model.h5'")

# Convert Hybrid to INT8
print("\nConverting Hybrid to INT8...")

def representative_dataset_hybrid():
    for i in range(100):
        yield [X_train_cnn[i:i+1].astype(np.float32)]

converter_hybrid = tf.lite.TFLiteConverter.from_keras_model(model_hybrid_f32)
converter_hybrid.optimizations = [tf.lite.Optimize.DEFAULT]
converter_hybrid.representative_dataset = representative_dataset_hybrid
converter_hybrid.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_hybrid.inference_input_type = tf.int8
converter_hybrid.inference_output_type = tf.int8

try:
    tflite_model_hybrid = converter_hybrid.convert()
    with open('hybrid_int8_model.tflite', 'wb') as f:
        f.write(tflite_model_hybrid)
    print("Hybrid INT8 model saved as 'hybrid_int8_model.tflite'")

    tflite_size_hybrid = len(tflite_model_hybrid) / 1024
    print(f"Hybrid INT8 Model size: {tflite_size_hybrid:.2f} KB")
except Exception as e:
    print(f"Error converting Hybrid to INT8: {e}")

# ============================================================================
# MODEL COMPARISON AND SUMMARY
# ============================================================================

print("\n" + "="*50)
print("MODEL TRAINING COMPLETED - SUMMARY")
print("="*50)

print(f"\nDataset Information:")
print(f"Total samples: {len(X)}")
print(f"Number of classes: {len(label_encoder.classes_)}")
print(f"Classes: {list(label_encoder.classes_)}")

print(f"\nModel Performance:")
print(f"CNN Model Test Accuracy:     {cnn_test_acc:.4f}")
print(f"MLP Model Test Accuracy:     {mlp_test_acc:.4f}")
print(f"Hybrid Model Test Accuracy:  {hybrid_test_acc:.4f}")

print(f"\nGenerated Models:")
print("1. cnn_float32_model.h5")
print("2. cnn_int8_model.tflite")
print("3. mlp_float32_model.h5")
print("4. mlp_int8_model.tflite")
print("5. hybrid_float32_model.h5")
print("6. hybrid_int8_model.tflite")

print(f"\nModel Sizes (INT8):")
print(f"CNN:     {tflite_size:.2f} KB")
print(f"MLP:     {tflite_size_mlp:.2f} KB")
print(f"Hybrid:  {tflite_size_hybrid:.2f} KB")

print(f"\nRecommended for ESP32:")
best_model = max(cnn_test_acc, mlp_test_acc, hybrid_test_acc)
if best_model == cnn_test_acc:
    print("✓ Use CNN model - Good accuracy with moderate complexity")
elif best_model == mlp_test_acc:
    print("✓ Use MLP model - Simple and efficient for ESP32")
else:
    print("✓ Use Hybrid model - Best accuracy, slightly larger")

# Save training summary
summary = {
    "dataset": {
        "total_samples": len(X),
        "train_samples": len(X_train),
        "val_samples": len(X_val),
        "test_samples": len(X_test),
        "classes": label_encoder.classes_.tolist()
    },
    "model_performance": {
        "cnn_accuracy": float(cnn_test_acc),
        "mlp_accuracy": float(mlp_test_acc),
        "hybrid_accuracy": float(hybrid_test_acc)
    },
    "model_sizes_kb": {
        "cnn_int8": float(tflite_size),
        "mlp_int8": float(tflite_size_mlp),
        "hybrid_int8": float(tflite_size_hybrid)
    }
}

with open("training_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\nTraining summary saved to 'training_summary.json'")
print("\nAll models trained successfully! ✅")

Loading data from CSV...
Processing features and labels...
Splitting data...
Saving processed data...

Dataset Statistics:
Total samples: 20000
Training samples: 14000
Validation samples: 3000
Test samples: 3000
Classes: ['SAWTOOTH' 'SINE' 'SQUARE' 'TRIANGLE']

Training CNN Float32 model...
Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 17s 63ms/step - accuracy: 0.9144 - loss: 0.3136 - val_accuracy: 0.2500 - val_loss: 5.4600 - learning_rate: 0.0010
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 13s 59ms/step - accuracy: 0.9966 - loss: 0.0171 - val_accuracy: 0.2593 - val_loss: 6.0174 - learning_rate: 0.0010
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.9980 - loss: 0.0122 - val_accuracy: 0.9800 - val_loss: 0.0563 - learning_rate: 0.0010
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.9994 - loss: 0.0044 - val_accuracy: 0.5793 - val_loss: 2.6599 - learning_rate: 0.0010
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 15s 68ms/step - accuracy: 0.9988 - loss: 0.0057 - va


CNN Test Accuracy: 0.9990
CNN Float32 model saved as 'cnn_float32_model.h5'

Converting CNN to INT8...
Saved artifact at '/tmp/tmpmw83na6e'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 1), dtype=tf.float32, name='keras_tensor_37')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  132365092273616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365111228368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364637797648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132367554926480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365092271696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364637797456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364637797264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364637798032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364637793040: TensorSpec(shape=(), dtype=tf.resou

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


CNN INT8 model saved as 'cnn_int8_model.tflite'
CNN INT8 Model size: 57.30 KB

Training MLP Float32 model...
Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6963 - loss: 0.7946 - val_accuracy: 0.8450 - val_loss: 0.5489 - learning_rate: 0.0010
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9529 - loss: 0.1573 - val_accuracy: 0.9173 - val_loss: 0.1978 - learning_rate: 0.0010
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9668 - loss: 0.1087 - val_accuracy: 0.9713 - val_loss: 0.0971 - learning_rate: 0.0010
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9732 - loss: 0.0853 - val_accuracy: 0.9987 - val_loss: 0.0342 - learning_rate: 0.0010
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9739 - loss: 0.0815 - val_accuracy: 0.9987 - val_loss: 0.0231 - learning_rate: 0.0010
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9765 - loss: 0.0754 - val_accuracy: 0.9987 - val_loss: 0.0193 - l


MLP Test Accuracy: 0.9990
MLP Float32 model saved as 'mlp_float32_model.h5'

Converting MLP to INT8...
Saved artifact at '/tmp/tmpjpytxcp0'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256), dtype=tf.float32, name='keras_tensor_50')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  132365096293904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096296208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365051623760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365051622992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096288528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365051623568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365051622800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365051624144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365051623376: TensorSpec(shape=(), dtype=tf.resource

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


219/219 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.8291 - loss: 0.5168 - val_accuracy: 0.4993 - val_loss: 2.4830 - learning_rate: 0.0010
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 0.9958 - loss: 0.0319 - val_accuracy: 0.5127 - val_loss: 3.2253 - learning_rate: 0.0010
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.9978 - loss: 0.0176 - val_accuracy: 0.7657 - val_loss: 0.5586 - learning_rate: 0.0010
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.9963 - loss: 0.0174 - val_accuracy: 0.7943 - val_loss: 0.4482 - learning_rate: 0.0010
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.9990 - loss: 0.0081 - val_accuracy: 0.9990 - val_loss: 0.0122 - learning_rate: 0.0010
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9979 - loss: 0.0116 - val_accuracy: 0.9807 - val_loss: 0.0729 - learning_rate: 0.0010
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.9983 - loss: 0.0096 - va


Hybrid Test Accuracy: 0.9990
Hybrid Float32 model saved as 'hybrid_float32_model.h5'

Converting Hybrid to INT8...
Saved artifact at '/tmp/tmpkot4kscr'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 1), dtype=tf.float32, name='keras_tensor_60')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  132364882858384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096298704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096301008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096296592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096301968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096302160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096294480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096303312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365096301584: TensorSpec(shape=(), dt

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
import tensorflow as tf
import numpy as np

print("\n" + "="*50)
print("Converting Float32 H5 models to Float32 TFLite...")
print("="*50)

# --- Convert CNN Float32 H5 to TFLite ---
print("\nConverting cnn_float32_model.h5 to TFLite...")
model_f32_cnn_h5 = tf.keras.models.load_model('cnn_float32_model.h5')
converter_cnn_f32 = tf.lite.TFLiteConverter.from_keras_model(model_f32_cnn_h5)
tflite_model_cnn_f32 = converter_cnn_f32.convert()

try:
    with open('cnn_float32_model.tflite', 'wb') as f:
        f.write(tflite_model_cnn_f32)
    print("CNN Float32 TFLite model saved as 'cnn_float32_model.tflite'")
    tflite_size_cnn_f32 = len(tflite_model_cnn_f32) / 1024
    print(f"CNN Float32 TFLite Model size: {tflite_size_cnn_f32:.2f} KB")
except Exception as e:
    print(f"Error converting CNN H5 to Float32 TFLite: {e}")

# --- Convert MLP Float32 H5 to TFLite ---
print("\nConverting mlp_float32_model.h5 to TFLite...")
model_f32_mlp_h5 = tf.keras.models.load_model('mlp_float32_model.h5')
converter_mlp_f32 = tf.lite.TFLiteConverter.from_keras_model(model_f32_mlp_h5)
tflite_model_mlp_f32 = converter_mlp_f32.convert()

try:
    with open('mlp_float32_model.tflite', 'wb') as f:
        f.write(tflite_model_mlp_f32)
    print("MLP Float32 TFLite model saved as 'mlp_float32_model.tflite'")
    tflite_size_mlp_f32 = len(tflite_model_mlp_f32) / 1024
    print(f"MLP Float32 TFLite Model size: {tflite_size_mlp_f32:.2f} KB")
except Exception as e:
    print(f"Error converting MLP H5 to Float32 TFLite: {e}")

# --- Convert Hybrid Float32 H5 to TFLite ---
print("\nConverting hybrid_float32_model.h5 to TFLite...")
model_f32_hybrid_h5 = tf.keras.models.load_model('hybrid_float32_model.h5')
converter_hybrid_f32 = tf.lite.TFLiteConverter.from_keras_model(model_f32_hybrid_h5)
tflite_model_hybrid_f32 = converter_hybrid_f32.convert()

try:
    with open('hybrid_float32_model.tflite', 'wb') as f:
        f.write(tflite_model_hybrid_f32)
    print("Hybrid Float32 TFLite model saved as 'hybrid_float32_model.tflite'")
    tflite_size_hybrid_f32 = len(tflite_model_hybrid_f32) / 1024
    print(f"Hybrid Float32 TFLite Model size: {tflite_size_hybrid_f32:.2f} KB")
except Exception as e:
    print(f"Error converting Hybrid H5 to Float32 TFLite: {e}")

print("\nFloat32 TFLite models generated successfully! ✅")



Converting Float32 H5 models to Float32 TFLite...

Converting cnn_float32_model.h5 to TFLite...
Saved artifact at '/tmp/tmpmp9k9kg2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 1), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  132365061252176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061258512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061257744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061253520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061257936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061259088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061257552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061253328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061253904: TensorSpec(shape=(), dtype=tf.resource, name

CNN Float32 TFLite model saved as 'cnn_float32_model.tflite'
CNN Float32 TFLite Model size: 165.36 KB

Converting mlp_float32_model.h5 to TFLite...
Saved artifact at '/tmp/tmpgvklamh_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256), dtype=tf.float32, name='input_layer_4')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  132365055115344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365055115536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866995216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866995600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061258128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132365061251984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866994448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866995024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1323648669946

MLP Float32 TFLite model saved as 'mlp_float32_model.tflite'
MLP Float32 TFLite Model size: 172.12 KB

Converting hybrid_float32_model.h5 to TFLite...
Saved artifact at '/tmp/tmpecnmka5y'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 1), dtype=tf.float32, name='input_layer_5')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  132364867000208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866999824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866598096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866597136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866999440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866597904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866597712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132364866598288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1323648